In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm

In [8]:
m = torch.tensor([1, 2, 3])
x = torch.tensor([1, 2])
b = torch.tensor([1, 2, 1])
# y = torch.tensor([3, 4])
x = torch.tensor(np.array([1, 2]))
y = torch.tensor(np.array([3, 4]))

mx = torch.einsum('m,x -> xm', m, x)
mxb = mx + b
print(mx)
print(mxb)
print(y - mxb.transpose(0, 1))
print((y - mxb.transpose(0, 1)))
print((y - mxb.transpose(0, 1)).sum(axis=1))

tensor([[1, 2, 3],
        [2, 4, 6]])
tensor([[2, 4, 4],
        [3, 6, 7]])
tensor([[ 1,  1],
        [-1, -2],
        [-1, -3]])
tensor([[ 1,  1],
        [-1, -2],
        [-1, -3]])
tensor([ 2, -3, -4])


C:\Users\pande\AppData\Local\Temp\ipykernel_5796\3177086203.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.tensor([1, 2]))


In [107]:
class Distribution(nn.Module):
    def __init__(self, size=1000):
        super().__init__()
        self.mu = nn.Parameter(torch.tensor(0.0), requires_grad=True)
        self.std = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.size = size
    def forward(self):
        return self.mu + self.std * torch.normal(0, 1, size=(1, self.size))

class LinearRegression(nn.Module):
    def __init__(self, size=1000):
        super().__init__()
        self.m = Distribution(size=size)
        self.b = Distribution(size=size)
    def forward(self, x, y):
        m = self.m()
        b = self.b()
        
        def loss(x_i, y_i, m_j, b_j):
            return (y_i - (m_j * x_i - b_j)) ** 2

        def loss_val(x_i, y_i):
            vals = np.vectorize(loss)(x_i, y_i, m, b)
            return (1 / len(m)) * np.sum(vals)
    
        vals = np.vectorize(loss_val)(x, y)
        total_loss = np.sum(vals) * (1/ len(x))
        
        # Minimize loss by maximizing this value
        return torch.tensor((total_loss), requires_grad=True)

In [108]:
def training(x, y, epochs=5):
    model = LinearRegression()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in tqdm(range(epochs), desc="Training..."):
        # Zeros gradiant for training
        optimizer.zero_grad()
        
        # Calculates likelihood
        loglik = model(x, y)
        e = torch.mean(loglik)
        
        # Updates parameters
        e.backward()
        optimizer.step()
        
    return model

In [109]:
df = pd.read_csv('data/Advertising.csv').drop('Unnamed: 0', axis=1)
df

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [110]:
x = df.TV.to_numpy()
y = df.Sales.to_numpy()

In [111]:
res = training(x, y, epochs=10)
res

Training...:   0%|          | 0/10 [00:00<?, ?it/s]

<class 'torch.Tensor'>


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
print(res.m.mu)

Parameter containing:
tensor(0., requires_grad=True)


In [ ]:
print(res.m.std)

Parameter containing:
tensor(1., requires_grad=True)


In [ ]:
print(res.b.mu)

Parameter containing:
tensor(0., requires_grad=True)


In [ ]:
print(res.b.std)

Parameter containing:
tensor(1., requires_grad=True)
